In [1]:
import pandas as pd
import time
from tqdm import tqdm, trange

In [4]:
def indexed_abstract_to_string(indexed_abstract):
    words = ["" for _ in range(indexed_abstract['IndexLength'])]
    for k, v in indexed_abstract['InvertedIndex'].items():
        for i in v: 
            words[i] = k
    return " ".join(words)

def authors_to_ids(authors):
    return [e['id'] for e in authors]

In [5]:
cols = ['id', 'title', 'indexed_abstract', 'authors', 'references']

for name in tqdm([ "LLL", "LLR", "LRL", "LRR", 
                   "RLL", "RLR", "RRL", "RRR"]):
    df = None
    df = pd.read_json(f"./data/archive/{name}.json")
    df = df[~df[cols].isna().any(axis=1)]
    df['abstract']   = df.indexed_abstract.map(indexed_abstract_to_string)
    df['author_ids'] = df.authors.map(authors_to_ids)
    df[['id', 'author_ids', 'title', 'abstract', 'references']].to_parquet(f'./data/{name}.parquet')

100%|█████████████████████████████████████████████████████████████████████████████████| 8/8 [19:28<00:00, 146.05s/it]


In [2]:
dfs = [pd.read_parquet(f'./data/{name}.parquet') for name in tqdm([ "LLL", "LLR", "LRL", "LRR", 
                                                                    "RLL", "RLR", "RRL", "RRR"])]

100%|██████████████████████████████████████████████████████████████████████████████████| 8/8 [00:34<00:00,  4.28s/it]


In [10]:
df = dfs[0]
for d in tqdm(dfs[1:]):
    df = df.append(d, ignore_index=True)

  0%|                                                                                          | 0/7 [00:00<?, ?it/s]/tmp/ipykernel_24541/2641661376.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d, ignore_index=True)
100%|██████████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  4.48it/s]


In [13]:
df.to_parquet("data.parquet")